## Plotting the qualitative data

First use calibration from Ga and As to make a table to see if the peaks deviate from the expected values.

Then plot the qualitative data of each sample place.

In [3]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import hyperspy.api as hs
from scipy.signal import find_peaks

# utils and helper_files
from utils.get_raw_data import get_array_and_names
from helper_files.normalizations import normalize_high_peak
from helper_files.peak_ratios import channel_to_kev, make_fit, n_gaussians_and_m_order_bg
from helper_files.calibration import calibrate_channel_width_two_peaks

In [4]:
ga, ga_names = get_array_and_names('GaAs')
mo, mo_names = get_array_and_names('Mo')
nw, nw_names = get_array_and_names('NW')
cu, cu_names = get_array_and_names('Cu') # mostly C
si, si_names = get_array_and_names('Si')
fe, fe_names = get_array_and_names('Fe') # it is Al not Fe

In [5]:
def calibrate_spectrum(spectrum, peaks_keV, initial_dispersion=0.01, initial_offset=20, deg=12, plot_fit=False):
    """
    Calibrate a spectrum using two peak positions.
    Finding the peaks automatically.
    Making a fit model with n gaussians and <deg> order background.
    Finding the closest peaks to the given peaks_keV.
    Then returning the dispersion and offset.
    """
    channels = np.arange(len(spectrum))
    # find peaks
    peaks_channel_est, _ = find_peaks(spectrum, prominence=0.01)
    peaks_keV_est = channel_to_kev(peaks_channel_est, dispersion=initial_dispersion, offset=initial_offset)

    # find the peaks that are closest to the peaks_keV
    peaks_to_fit_index = []
    for peak in peaks_keV:
        peaks_to_fit_index.append(np.argmin(np.abs(peaks_keV_est - peak)))

    # fit with make_fit
    # make_fit(array, x, deg=12, prominence=0.01, pixel_removal=50, offset=20) >>> fit_vals, covar
    fit_vals, _ = make_fit(spectrum, channels, deg=deg)


    fit_peaks = fit_vals[deg + 2::3]
    dispersion, offset = calibrate_channel_width_two_peaks(fit_peaks[peaks_to_fit_index], peaks_keV)
    fit_peaks_keV = channel_to_kev(fit_peaks, dispersion=dispersion, offset=offset)
    keV = channel_to_kev(channels, dispersion=dispersion, offset=offset)
    if plot_fit:
        fit = n_gaussians_and_m_order_bg(channels, deg, *fit_vals)
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=keV, y=spectrum, name='spectrum', mode='markers'))
        fig.add_trace(go.Scatter(x=keV, y=fit, name='fit'))
        fig.add_vline(x=fit_peaks_keV[peaks_to_fit_index[0]], line_width=2, line_dash="dash")
        fig.add_vline(x=fit_peaks_keV[peaks_to_fit_index[1]], line_width=2, line_dash="dash")
        fig.update_layout(title='Calibration of the spectrum', xaxis_title='Energy [keV]', yaxis_title='Relative intensity')
        fig.show()
    
    return dispersion, offset

def find_nearest(array, value, max_deviation=0.05):
    """
    Finds the index of the value in array that is closest to value.
    If the deviation is larger than max_deviation in percent, returns None.

    Parameters
    ----------
    array : array of floats
        peak array
    value : float
        float to find the closest value to
    max_deviation : float, optional
        max deviation in percent, by default 0.05

    Returns
    -------
    index : int or None
        Index of the closest value in array to value. None if the deviation is larger than max_deviation.
    """
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    # check if the deviation is larger than max_deviation in percent
    if np.abs(array[idx] - value) / value > max_deviation:
        print(f'Warning: {array[idx]} is not within {max_deviation * 100}% of {value}')
        return idx
    return idx

def peak_deviation(peaks_energy, peaks_fit):
    """
    Finds the corresponding peaks in peaks_fit to peaks_energy,
    and returns the deviation in percent.
    """
    peaks_energy = np.array(peaks_energy)
    peaks_fit_selected = []
    for p in peaks_energy:
        index = find_nearest(peaks_fit, p)
        if index is not None:
            peaks_fit_selected.append(peaks_fit[index])

    peaks_fit_selected = np.array(peaks_fit_selected)
    # deviation = np.abs(peaks_fit_selected - peaks_energy) / peaks_energy * 100
    deviation = (peaks_fit_selected - peaks_energy) / peaks_energy * 100

    return deviation, peaks_fit_selected


In [6]:
peaks_calibration = [1.098, 10.5436]
print(f'Calibrating on {ga_names[-1]}, with peaks {peaks_calibration}')
dispersion, offset = calibrate_spectrum(ga[-1], peaks_calibration, plot_fit=False)


Calibrating on GaAs_30kV, with peaks [1.098, 10.5436]
The calibration factor is: 0.0100300 keV/channel, with 21.127 channels zero offset


In [7]:
# figure out how good the calibration is on other peaks
def calibration_check_on_peaks(spectrum, elements, lines, vacc, dispersion, offset):
    """
    Checks the calibration on a spectrum.
    Quantifies this by the deviation of the peak positions from the expected positions.

    Parameters
    ----------
    spectrum : array of floats
        intensity values
    elements : array of strings
        elements to check
    lines : array of strings
        lines to check
    vacc : int
        acceleration voltage
    dispersion : float
        dispersion of the calibration
    offset : float
        offset of the calibration in channels
    
    Returns
    -------
    deviation : array of floats
        deviation of the peak positions from the expected positions
    peaks_fit : array of floats
        peak positions of the fit, only the selected peaks
    peaks_energy : array of floats
        expected peak positions, from HyperSpy
    peaks_names : array of strings
        names of the peaks
    """    
    peaks_energy = []
    peaks_names = []
    for e in elements:
        for l in lines:
            try:
                energy = float(hs.material.elements[e].Atomic_properties['Xray_lines'][l]['energy (keV)'])
                if energy < vacc:
                    peaks_energy.append(energy)
                    peaks_names.append(f'{e} {l}')
                else:
                    print(f'{e} {l} at {energy} is above E_0 = {vacc} kV')
                if energy > vacc / 2:
                    print(f'Warning: {e:<2} {l:<2} at {energy:<8} have overvoltage below 2')
            except KeyError:
                pass
    print(f'Checking peaks: {peaks_names} at {peaks_energy}, with Vacc = {vacc} kV')

    channels = np.arange(len(spectrum))
    # keV = channel_to_kev(channels, dispersion=dispersion, offset=offset)
    fit_vals, _ = make_fit(spectrum, channels, deg=12)
    fit_peaks = fit_vals[12 + 2::3]
    fit_peaks_kev = channel_to_kev(fit_peaks, dispersion=dispersion, offset=offset)

    dev, peaks_fit_selected = peak_deviation(peaks_energy, fit_peaks_kev)
    print(dev)
    for i in range(len(dev)):
        print(f'{peaks_names[i]:<6} {peaks_fit_selected[i]:<8.4f} deviates {dev[i]:.3f}% from {peaks_energy[i]}')

    return dev, peaks_fit_selected, peaks_energy, peaks_names


In [8]:
ga30 = calibration_check_on_peaks(ga[-1], ['As', 'Ga'], ['Ka', 'La'], 30, dispersion, offset)
ga15 = calibration_check_on_peaks(ga[-2], ['As', 'Ga'], ['Ka', 'La'], 15, dispersion, offset)
ga10 = calibration_check_on_peaks(ga[-3], ['As', 'Ga'], ['La'], 10, dispersion, offset)
ga05 = calibration_check_on_peaks(ga[0], ['As', 'Ga'], ['La'], 5, dispersion, offset)


Checking peaks: ['As Ka', 'As La', 'Ga Ka', 'Ga La'] at [10.5436, 1.2819, 9.2517, 1.098], with Vacc = 30 kV
[0.         0.3684714  0.02364178 0.        ]
As Ka  10.5436  deviates 0.000% from 10.5436
As La  1.2866   deviates 0.368% from 1.2819
Ga Ka  9.2539   deviates 0.024% from 9.2517
Ga La  1.0980   deviates 0.000% from 1.098
Checking peaks: ['As Ka', 'As La', 'Ga Ka', 'Ga La'] at [10.5436, 1.2819, 9.2517, 1.098], with Vacc = 15 kV
[-0.06100983  0.37623324  0.02257412  0.12921206]
As Ka  10.5372  deviates -0.061% from 10.5436
As La  1.2867   deviates 0.376% from 1.2819
Ga Ka  9.2538   deviates 0.023% from 9.2517
Ga La  1.0994   deviates 0.129% from 1.098
Checking peaks: ['As La', 'Ga La'] at [1.2819, 1.098], with Vacc = 10 kV
[0.39715446 0.195558  ]
As La  1.2870   deviates 0.397% from 1.2819
Ga La  1.1001   deviates 0.196% from 1.098
Checking peaks: ['As La', 'Ga La'] at [1.2819, 1.098], with Vacc = 5 kV
[0.41680165 0.26947321]
As La  1.2872   deviates 0.417% from 1.2819
Ga La  1.10

In [9]:
# see lines
hs.material.elements['Fe'].Atomic_properties['Xray_lines']

├── Ka
│   ├── energy (keV) = 6.4039
│   └── weight = 1.0
├── Kb
│   ├── energy (keV) = 7.058
│   └── weight = 0.1272
├── La
│   ├── energy (keV) = 0.7045
│   └── weight = 1.0
├── Lb3
│   ├── energy (keV) = 0.7921
│   └── weight = 0.02448
├── Ll
│   ├── energy (keV) = 0.6152
│   └── weight = 0.3086
└── Ln
    ├── energy (keV) = 0.6282
    └── weight = 0.12525

In [10]:
nw30 = calibration_check_on_peaks(nw[-1], ['As', 'Ga', 'Cu', ], ['La', 'Ka',], 30, dispersion, offset)

Checking peaks: ['As La', 'As Ka', 'Ga La', 'Ga Ka', 'Cu La', 'Cu Ka'] at [1.2819, 10.5436, 1.098, 9.2517, 0.9295, 8.0478], with Vacc = 30 kV
[ 0.42177766 -0.00999266  0.31793251  0.02423443  0.85669511  0.04549944]
As La  1.2873   deviates 0.422% from 1.2819
As Ka  10.5425  deviates -0.010% from 10.5436
Ga La  1.1015   deviates 0.318% from 1.098
Ga Ka  9.2539   deviates 0.024% from 9.2517
Cu La  0.9375   deviates 0.857% from 0.9295
Cu Ka  8.0515   deviates 0.045% from 8.0478


In [11]:
dispersion_AZ = 0.01
offset_AZ = 20
nw30_AZ = calibration_check_on_peaks(nw[-1], ['As', 'Ga', 'Cu', ], ['La', 'Ka',], 30, dispersion_AZ, offset_AZ)

Checking peaks: ['As La', 'As Ka', 'Ga La', 'Ga Ka', 'Cu La', 'Cu Ka'] at [1.2819, 10.5436, 1.098, 9.2517, 0.9295, 8.0478], with Vacc = 30 kV
[ 1.0002528  -0.20229805  1.04392777 -0.15325131  1.76709794 -0.11383186]
As La  1.2947   deviates 1.000% from 1.2819
As Ka  10.5223  deviates -0.202% from 10.5436
Ga La  1.1095   deviates 1.044% from 1.098
Ga Ka  9.2375   deviates -0.153% from 9.2517
Cu La  0.9459   deviates 1.767% from 0.9295
Cu Ka  8.0386   deviates -0.114% from 8.0478


In [12]:
0.211377  / 0.010028

21.078679696848823

In [13]:
# # # if I use calibration from model fit in HyperSpy, Ga Ka_Calibrated center_ is 9.2525 and not 9.2528
# HyperSpy modelfit of 30 kV: dispersion = 0.010028, offset = 0.211377  / 0.010028

dispersion_HS = 0.010028
offset_HS =  0.211377  / 0.010028
nw30_HS = calibration_check_on_peaks(nw[-1], ['As', 'Ga', 'Cu', ], ['La', 'Ka',], 30, dispersion_HS, offset_HS)

Checking peaks: ['As La', 'As Ka', 'Ga La', 'Ga Ka', 'Cu La', 'Cu Ka'] at [1.2819, 10.5436, 1.098, 9.2517, 0.9295, 8.0478], with Vacc = 30 kV
[ 0.43922793 -0.02545753  0.34169594  0.00940054  0.88830187  0.03144001]
As La  1.2875   deviates 0.439% from 1.2819
As Ka  10.5409  deviates -0.025% from 10.5436
Ga La  1.1018   deviates 0.342% from 1.098
Ga Ka  9.2526   deviates 0.009% from 9.2517
Cu La  0.9378   deviates 0.888% from 0.9295
Cu Ka  8.0503   deviates 0.031% from 8.0478


In [14]:
mo30 = calibration_check_on_peaks(mo[-1], ['Mo'], ['Ka','La'], 30, dispersion, offset)
mo30_AZ = calibration_check_on_peaks(mo[-1], ['Mo'], ['Ka','La'], 30, dispersion_AZ, offset_AZ)
mo30_HS = calibration_check_on_peaks(mo[-1], ['Mo'], ['Ka','La'], 30, dispersion_HS, offset_HS)

Checking peaks: ['Mo Ka', 'Mo La'] at [17.4793, 2.2932], with Vacc = 30 kV
[-0.09047494  0.85778642]
Mo Ka  17.4635  deviates -0.090% from 17.4793
Mo La  2.3129   deviates 0.858% from 2.2932
Checking peaks: ['Mo Ka', 'Mo La'] at [17.4793, 2.2932], with Vacc = 30 kV
[-0.32494166  1.04734763]
Mo Ka  17.4225  deviates -0.325% from 17.4793
Mo La  2.3172   deviates 1.047% from 2.2932
Checking peaks: ['Mo Ka', 'Mo La'] at [17.4793, 2.2932], with Vacc = 30 kV
[-0.10773613  0.85858127]
Mo Ka  17.4605  deviates -0.108% from 17.4793
Mo La  2.3129   deviates 0.859% from 2.2932


In [15]:
si30 = calibration_check_on_peaks(si[-1], ['Si'], ['Ka'], 30, dispersion, offset)
si30_AZ = calibration_check_on_peaks(si[-1], ['Si'], ['Ka'], 30, dispersion_AZ, offset_AZ)
si30_HS = calibration_check_on_peaks(si[-1], ['Si'], ['Ka'], 30, dispersion_HS, offset_HS)

Checking peaks: ['Si Ka'] at [1.7397], with Vacc = 30 kV
[-0.18246153]
Si Ka  1.7365   deviates -0.182% from 1.7397
Checking peaks: ['Si Ka'] at [1.7397], with Vacc = 30 kV
[0.16653105]
Si Ka  1.7426   deviates 0.167% from 1.7397
Checking peaks: ['Si Ka'] at [1.7397], with Vacc = 30 kV
[-0.17477654]
Si Ka  1.7367   deviates -0.175% from 1.7397


In [16]:
# on Fe, which is really Al
al30 = calibration_check_on_peaks(fe[0], ['Al'], ['Ka'], 30, dispersion, offset)
al30_AZ = calibration_check_on_peaks(fe[0], ['Al'], ['Ka'], 30, dispersion_AZ, offset_AZ)
al30_HS = calibration_check_on_peaks(fe[0], ['Al'], ['Ka'], 30, dispersion_HS, offset_HS)

Checking peaks: ['Al Ka'] at [1.4865], with Vacc = 30 kV
[-0.25941194]
Al Ka  1.4826   deviates -0.259% from 1.4865
Checking peaks: ['Al Ka'] at [1.4865], with Vacc = 30 kV
[0.20012592]
Al Ka  1.4895   deviates 0.200% from 1.4865
Checking peaks: ['Al Ka'] at [1.4865], with Vacc = 30 kV
[-0.2469962]
Al Ka  1.4828   deviates -0.247% from 1.4865


In [17]:
# on Cu, which is mostly C
c30 = calibration_check_on_peaks(cu[-1], ['Cu', 'C'], ['Ka'], 30, dispersion, offset)
c30_AZ = calibration_check_on_peaks(cu[-1], ['Cu', 'C'], ['Ka'], 30, dispersion_AZ, offset_AZ)
c30_HS = calibration_check_on_peaks(cu[-1], ['Cu', 'C'], ['Ka'], 30, dispersion_HS, offset_HS)

Checking peaks: ['Cu Ka', 'C Ka'] at [8.0478, 0.2774], with Vacc = 30 kV
[ 0.04347143 -6.73768165]
Cu Ka  8.0513   deviates 0.043% from 8.0478
C Ka   0.2587   deviates -6.738% from 0.2774
Checking peaks: ['Cu Ka', 'C Ka'] at [8.0478, 0.2774], with Vacc = 30 kV
[-0.11585381 -2.95505631]
Cu Ka  8.0385   deviates -0.116% from 8.0478
C Ka   0.2692   deviates -2.955% from 0.2774
Checking peaks: ['Cu Ka', 'C Ka'] at [8.0478, 0.2774], with Vacc = 30 kV
[ 0.0294124  -6.58275369]
Cu Ka  8.0502   deviates 0.029% from 8.0478
C Ka   0.2591   deviates -6.583% from 0.2774


In [22]:
hspy = nw30_HS
az = nw30_AZ
my = nw30

hspy2 = mo30_HS
az2 = mo30_AZ
my2 = mo30

hspy3 = si30_HS
az3 = si30_AZ
my3 = si30

hspy4 = al30_HS
az4 = al30_AZ
my4 = al30

hspy5 = c30_HS
az5 = c30_AZ
my5 = c30

hyper = [hspy, hspy2, hspy3, hspy4, hspy5]
aztec = [az, az2, az3, az4, az5]
mycalib = [my, my2, my3, my4, my5]

alpha = '$\\alpha$'

print(f'{"Peak":<20} & {"Theoretical [keV]":<20} & {"AZ dev. [%]":<15} & {"HS dev. [%]":<15} & {"My dev. [%]":<15} \\\\')

for i in range(len(hyper)):
    for j in range(len(hyper[i][0])):
        print(f'{hyper[i][3][j][:-1]+alpha:<20} & {hyper[i][2][j]:<20} & {aztec[i][0][j]:<15.3f} & {hyper[i][0][j]:<15.3f} & {mycalib[i][0][j]:<15.3f} \\\\')


Peak                 & Theoretical [keV]    & AZ dev. [%]     & HS dev. [%]     & My dev. [%]     \\
As L$\alpha$         & 1.2819               & 1.000           & 0.439           & 0.422           \\
As K$\alpha$         & 10.5436              & -0.202          & -0.025          & -0.010          \\
Ga L$\alpha$         & 1.098                & 1.044           & 0.342           & 0.318           \\
Ga K$\alpha$         & 9.2517               & -0.153          & 0.009           & 0.024           \\
Cu L$\alpha$         & 0.9295               & 1.767           & 0.888           & 0.857           \\
Cu K$\alpha$         & 8.0478               & -0.114          & 0.031           & 0.045           \\
Mo K$\alpha$         & 17.4793              & -0.325          & -0.108          & -0.090          \\
Mo L$\alpha$         & 2.2932               & 1.047           & 0.859           & 0.858           \\
Si K$\alpha$         & 1.7397               & 0.167           & -0.175          & -0.182   